<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/7C_%E6%8C%87%E6%A8%99%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB_tfidf%E3%81%A7%E9%A0%BB%E5%87%BA%E5%8D%98%E8%AA%9E%E3%81%AE%E8%AA%BF%E6%9F%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>作成した分類モデルを活用してtfidfで分類に影響の強い単語の調査</center>

In [ ]:
# ISTPが0
# ENFJが1

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/研究室/研究_M2/src

In [ ]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  #print(DATETIME_now)

  return DATETIME_now

In [ ]:
#TYPE_INDEX='EorI_E'
#TYPE_INDEX='NorS_N'
#TYPE_INDEX='ForT_F'
TYPE_INDEX='JorP_J'

In [ ]:
INPUT_CSV_v7='./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_2021-12-04_W20R10000.csv'
INPUT_CSV_v7='./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_2022-01-25_W20R10000.csv'

In [ ]:
LEANING_RATE=3e-7
BATCH_SIZE=16
EPOCH=30

In [ ]:
SEASONS='S12ver3'
WORDS = 20
RAFFLE=30000

SWITCH_CONTINUE ='off'
#WITCH_CONTINUE ='on'

#model_S10ver2_L2e-07_B16_E20_W20_R30000_EorI_E.tar

In [ ]:
TERM='%s_L%s_B%s_E%s_W%s_R%s' % (SEASONS, LEANING_RATE, BATCH_SIZE, EPOCH, WORDS, RAFFLE)

print(TERM)

S12ver3_L3e-07_B16_E30_W20_R30000


In [ ]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

Defaulting to user installation because normal site-packages is not writeable
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/sh: 1: aptitude: not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable




---


In [ ]:
import pandas as pd
df=pd.read_csv('%s' % INPUT_CSV_v7, engine='python')
# データの確認
print(f'データサイズ： {df.shape}')
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].sample(10)

df.dropna(how='any', axis=0, inplace=True)
print(f'データサイズ： {df.shape}')

df=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df[['text', 'tweet_time', 'MBTI_Type','label_EorI_E','label_NorS_N','label_ForT_F','label_JorP_J']]

データサイズ： (10000, 15)
データサイズ： (10000, 15)


,text,tweet_time,MBTI_Type,label_EorI_E,label_NorS_N,label_ForT_F,label_JorP_J
0,半径5km以内に民家もコンビニも無いよーなトコで徒歩で放り出されてたからねw 1番近いコンビ...,2017-12-01 13:27:21+00:00,ISTP,0,0,0,0
1,で、重たい頭で目覚めた今日は、午後から海辺に出て焚き火してきた。次は山に篭りたいなぁ。,2017-12-03 13:16:09+00:00,ISFJ,0,0,1,1
2,ff外から失礼します、リプで貼りたいのであえて名前は出しませんが自分のストックやりませんか？...,2017-12-02 16:14:39+00:00,INFP,0,1,1,0
3,チロちゃんもかわいいですねー！ (*´∀｀*) チョコはキジトラとはちょっと違うの、かな？(...,2017-11-18 10:09:15+00:00,ISFJ,0,0,1,1
4,フォロワーさんに聞いてみよう モキュ猫の印象は？ 1 ドS 2 オタク 3 頭良い 4 い...,2017-02-09 03:39:03+00:00,INFP,0,1,1,0
...,...,...,...,...,...,...,...
9995,1/6(土)、東京の羽村市で中野七頭舞が披露されるそうで!щ(゜▽゜щ),2017-12-22 10:02:59+00:00,INFP,0,1,1,0
9996,というか練習ボイコットなんかしなくても交渉はまとまってたでしょ。自分の品位を無駄に落としただ...,2017-09-04 23:13:52+00:00,INFJ,0,1,1,1
9997,違和感をうまく言葉にできる方はすごい……ついーと拝見してぶんぶん頷いてる,2017-10-31 10:35:07+00:00,ISTJ,0,0,0,1
9998,内定者懇談会楽しかったな〜 人生哲学について社長とずっと話しててめちゃくちゃお酒が進んだw ...,2017-12-28 15:45:20+00:00,INFP,0,1,1,0


In [ ]:
sentences= df['text'].values #sentences = df.sentence.values
labels= df['label_%s' % TYPE_INDEX].values #labels = df.label.values

## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


from tqdm.notebook import tqdm
import numpy as np

# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)

list_word=[]
list_word.clear()
for sent in tqdm(sentences):
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # print(sent)
    # print(token_words)
    # print(type(token_words))
    # for element in token_words:
    #   print(element)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
len_max=max(max_len)
#len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
#len_q75=int(len_q75)
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')


最大単語数:  164
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = len_max,        # 文章の長さを固定（Padding/Trancatinating）#mayo
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                        truncation=True,
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# tenosor型に変換
labels = torch.tensor(labels)

print(input_ids)
print(attention_masks)
print(labels)
# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

/home/20VR016F/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



tensor([[    2, 10443,    76,  ...,     0,     0,     0],
        [    2,    12,     6,  ...,     0,     0,     0],
        [    2,  1044, 28890,  ...,     0,     0,     0],
        ...,
        [    2, 21650,    11,  ...,     0,     0,     0],
        [    2, 18085,   104,  ...,     0,     0,     0],
        [    2,  1325,     5,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 1, 0,  ..., 1, 0, 0])




---



In [ ]:
# for element1, element2, element3 in dataset:
#   print(element1)
#   print(element2)
#   print(element3)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
print("./model_%s_%s.tar" % (TERM, TYPE_INDEX))
checkpoint = torch.load("./model_%s_%s.tar" % (TERM, TYPE_INDEX))
model.load_state_dict(checkpoint["model_state_dict"])

./model_S12ver3_L3e-07_B16_E30_W20_R30000_JorP_J.tar


<All keys matched successfully>

In [ ]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install torchviz

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from tqdm.notebook import tqdm

model.eval()# 訓練モードをオフ

# 検証データローダー
preds_dataloader = DataLoader(
            dataset, 
            sampler = SequentialSampler(dataset), # 順番にデータを取得してバッチ化
            batch_size = 1
        )

list_preds=[]
list_preds.clear()
list_b_labels=[]
list_b_labels.clear()
for batch in tqdm(preds_dataloader):
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  list_b_labels.append(b_labels[0].item())
  #print(len(b_input_ids))
  #print(len(b_input_mask))
  #print(b_labels.values)
  with torch.no_grad():   
      # 学習済みモデルによる予測結果をpredsで取得     
      preds = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
      #print(preds[0].cpu().numpy()[0])
      list_preds.append(preds[0].cpu().numpy()[0])

In [ ]:
# STC ='今日は晴れている。'
# ans = model(STC)
# ans_index=np.argmax(ans[0])
# print('あなたの表情は', classes[ans_index], 'です。')



---



In [ ]:
# print(list_b_labels)
# for idx, element in enumerate(list_b_labels):
#   list_b_labels[idx]=element.item()
# print(list_b_labels)

In [ ]:
list_sentences=[]
list_sentences.clear()
print(sentences)
for idx, element in enumerate(sentences):
  #list_element[idx]=element.item()
  list_sentences.append(element)

['半径5km以内に民家もコンビニも無いよーなトコで徒歩で放り出されてたからねw 1番近いコンビニが多分5~6kmで、1番近いタクシー拾える所が多分8~10kmだから、歩くの無理な場所だったねｆ^^*) 放屁ワロタwwwww そりゃ乗せたくねーわなw'
 'で、重たい頭で目覚めた今日は、午後から海辺に出て焚き火してきた。次は山に篭りたいなぁ。'
 'ff外から失礼します、リプで貼りたいのであえて名前は出しませんが自分のストックやりませんか？ ふと目についたので聞いた次第です。' ...
 '違和感をうまく言葉にできる方はすごい……ついーと拝見してぶんぶん頷いてる'
 '内定者懇談会楽しかったな〜 人生哲学について社長とずっと話しててめちゃくちゃお酒が進んだw 同期と二次会にも行けて満足V'
 '私の性格タイプは “広報運動家” です。あなたのタイプは何ですか？ #16Personalities']


In [ ]:
list_probs=[]
list_probs.clear()
for i in range(0, len(list_preds)):
  list_probs.append(np.nan)
#print(list_probs)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # print(x)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

for idx, element in enumerate(list_preds):
  list_probs[idx]=softmax(element)
  # print()

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd

# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(list_preds, columns=['logit_0', 'logit_1']) #1はEN
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(list_preds, axis=1), columns=['pred_label'])
label_df = pd.DataFrame(list_b_labels, columns=['true_label'])
text_df  = pd.DataFrame(list_sentences, columns=['text'])
probs_df = pd.DataFrame(list_probs, columns=['prob_0', 'prob_1']) #1はEN
accuracy_df = pd.concat([text_df, logits_df, pred_df, label_df, probs_df], axis=1)
# accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()

accuracy_df.to_csv('./V7_%s_%s.csv' % (TERM, TYPE_INDEX))
print(('./V7_%s_%s.csv' % (TERM, TYPE_INDEX)))

./V7_S12ver3_L3e-07_B16_E30_W20_R30000_JorP_J.csv


In [ ]:
df_prob_0 = accuracy_df.sort_values('prob_0', ascending=False)
df_prob_1= accuracy_df.sort_values('prob_1', ascending=False)

In [ ]:
from IPython.display import HTML
#df_prob_0.head(20)
HTML(df_prob_0.head(10).to_html())

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
8490,ほんっっっつまもう仕事したくない起きたくない働きたくないいいいいいいいい くるしい,0.640270,-0.632289,0,0,0.781181,0.218819
5335,軌跡がほしい 自分が残ったって何にもならないのだけど ただ在るということが 在ることが苦しくなる時だってたくさんある それでも在ることに変わりがない それなら ただ在るという軌跡を ここに在る それだけを,0.616413,-0.466553,0,0,0.747055,0.252945
3319,うああああああああ！！！わかんないいいいいいいいい！！！！いやーーー！！！！！,0.578883,-0.452069,0,0,0.737100,0.262900
9160,こういうことしたいんですけどどうしたらいいですか？っていうのと こういうことするんですけどどうしたらいいですか？って全然違うよね,0.480659,-0.526624,0,1,0.732488,0.267512
7551,ニンジャ草原のイベクエわいわいやりたいけど最近みんな色々違うことやってるしなあ ゾンビもきょうはなさそうだ,0.520355,-0.480079,0,0,0.731144,0.268856
6804,生きている 生きてはいない 生きようかな 生きるのよそうかな,0.437694,-0.557440,0,0,0.730101,0.269899
8797,あ？？？！！ま？？！！！あええ？！チートラ最終話？？？？！！,0.532421,-0.462347,0,0,0.730029,0.269971
9338,普通に学校あるんですw マジでジワるw w w w w,0.625852,-0.343252,0,0,0.724941,0.275059
3883,きたああああああああああああああああああああああああああああああああああああああああああああ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！,0.540937,-0.421140,0,0,0.723538,0.276462
4170,今すぐ帰ってきたイカしばきたい 攻撃しちゃダメなタイミングがなんとなくわかったからもっとイカ行きたいいいいいいいい,0.493285,-0.461043,0,0,0.721985,0.278015


In [ ]:
from IPython.display import HTML
#df_prob_0.head(20)
HTML(df_prob_1.head(10).to_html())

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
7406,#FEヒーローズ #FEH #CM #偶然任せの未来でいいのか #やっぱつれぇわ #アクアは小室菜美の自己投影 #ドッカンテーブル,-0.475856,0.541942,1,1,0.265457,0.734543
4663,#FEヒーローズ #FEH #CM #偶然任せの未来でいいのか #やっぱつれぇわ #アクアは小室菜美の自己投影 #ドッカンテーブル,-0.475856,0.541942,1,1,0.265457,0.734543
4514,おはようございます、司令官。現在、ルート725630を進軍中です。今日も頑張りましょう！ #ミリ姫大戦RELOAD #にじよめ,-0.443542,0.530300,1,1,0.274115,0.725885
4613,おはようございます、司令官。現在、ルート847919を進軍中です。今日も頑張りましょう！ #ミリ姫大戦RELOAD #にじよめ,-0.452278,0.510377,1,1,0.276347,0.723653
5540,#ライアン・ガンダー 展 面白かった #monochrome #fujifilm #fujifilm_xseries #museum #japan #osaka #art #snapshot…,-0.447321,0.510168,1,0,0.277381,0.722619
9313,おはようございます、司令官。現在、ルート725596を進軍中です。今日も頑張りましょう！ #ミリ姫大戦RELOAD #にじよめ,-0.450902,0.504970,1,1,0.277705,0.722295
1545,Hi-STANDARD、18年ぶりアルバム「THE GIFT」発売へ - LINE NEWS #linenews,-0.479933,0.456068,1,0,0.281709,0.718291
1193,若葉 が LOL -lots of laugh-（EXTREME）の グレートクリアトライアル 成功！ #DIVANET,-0.456824,0.471204,1,1,0.283325,0.716675
6093,《毎日が》タイムセール♪ チャンスは6時間のみ！ 事前リサーチが出来るので余裕を持って仕入れましょう → #タイムセール #電脳せどり #楽天,-0.368692,0.520342,1,1,0.291309,0.708691
7974,#LINE桜くじ もらったよ！ ステキなご縁をありがとう。 #NYでお花見したい,-0.465269,0.421937,1,0,0.291687,0.708313




---



In [ ]:
accuracy_df = pd.concat([text_df, logits_df, pred_df, label_df, probs_df], axis=1)
# accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
0,半径5km以内に民家もコンビニも無いよーなトコで徒歩で放り出されてたからねw 1番近いコンビ...,-0.041569,-0.015783,1,0,0.493554,0.506446
1,で、重たい頭で目覚めた今日は、午後から海辺に出て焚き火してきた。次は山に篭りたいなぁ。,-0.269826,0.273062,1,1,0.367516,0.632484
2,ff外から失礼します、リプで貼りたいのであえて名前は出しませんが自分のストックやりませんか？...,0.009500,-0.030756,0,0,0.510063,0.489937
3,チロちゃんもかわいいですねー！ (*´∀｀*) チョコはキジトラとはちょっと違うの、かな？(...,0.075091,0.128998,1,1,0.486527,0.513473
4,フォロワーさんに聞いてみよう モキュ猫の印象は？ 1 ドS 2 オタク 3 頭良い 4 い...,0.343837,-0.391158,0,0,0.675900,0.324100


In [ ]:
test_df=accuracy_df
list_predLabel=test_df['pred_label'].to_list()
list_trureLabel = test_df['true_label'].to_list()

from sklearn.metrics import accuracy_score
print(accuracy_score(list_trureLabel,  list_predLabel))

from sklearn.metrics import precision_score
print(precision_score(list_trureLabel,  list_predLabel)) #適合率　#適合率（precision）は、陽性と予測されたサンプルのうち正解したサンプルの割合。

from sklearn.metrics import recall_score
print(recall_score(list_trureLabel,  list_predLabel)) #再現率 #再現率（recall）は実際に陽性のサンプルのうち正解したサンプルの割合。


list_predScore = test_df['logit_1']
def sigmoid(x):
    return 1/(1 + np.exp(-x))

list_predScore2=[]
list_predScore2.clear()
for idx, element in enumerate(list_predScore):
  list_predScore2.append(sigmoid(list_predScore[idx]))

# print(list_trureLabel)
# print(list_predScore2)

import numpy as np
from sklearn.metrics import roc_auc_score
roc_auc_score(list_trureLabel,  list_predScore2)

0.5185
0.39963774082002307
0.674916573971079


0.5637477211128473

In [ ]:
accuracy_df[accuracy_df['pred_label']==0]

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
2,ff外から失礼します、リプで貼りたいのであえて名前は出しませんが自分のストックやりませんか？...,0.009500,-0.030756,0,0,0.510063,0.489937
4,フォロワーさんに聞いてみよう モキュ猫の印象は？ 1 ドS 2 オタク 3 頭良い 4 い...,0.343837,-0.391158,0,0,0.675900,0.324100
5,…でも「藤原レイニー」は絶対ダメでしょ！（笑） 哲也は消したらダメだし… レイニー哲也もだめ...,0.117914,-0.005887,0,0,0.530911,0.469089
10,しょりぴのぞえり推しなのめっちゃ…たぎる… みゅーずの巨乳TOP2です…甘えられるお姉ちゃん...,0.273755,-0.103219,0,0,0.593143,0.406857
11,胸の内で楽しませていただきますので！どんどんお願いします(*•̀ᴗ•́*)و ̑̑,0.099328,0.084886,0,0,0.503610,0.496390
...,...,...,...,...,...,...,...
9982,君 鳥取まで出張して 私の代わりにご飯作って洗濯して掃除してくれたりしない？？もう私はベッド...,0.159486,-0.027020,0,0,0.546492,0.453508
9985,人生辛いことばっかなんだから コーヒーぐらい甘くていい。 この言葉大好きなので今日もこいつ...,0.070266,-0.036050,0,1,0.526554,0.473446
9990,お金の怖さ教えます！青木雄二先生の『ナニワ金融道』全巻がアプリで今だけ読める！,0.145952,-0.082895,0,0,0.556964,0.443036
9991,7月3日(月)までの期間限定！小山ゆう先生の『あずみ』全巻がアプリで今だけ読める！,0.059682,-0.062781,0,0,0.530578,0.469422


In [ ]:
# df_text_0= accuracy_df[accuracy_df['pred_label']==0]
list_text_0= df_prob_0.head(1000)['text'].values
# df_text_1= accuracy_df[accuracy_df['pred_label']==1]
list_text_1= df_prob_1.head(1000)['text'].values

list_text_all= df_prob_0['text'].values

In [ ]:
import numpy as np
import pandas as pd

def Bag_of_Words(morphemes): # 単語に数値を割り当てます。
    word2id = {} # {単語: ID}
    for line in morphemes:
        for word in line.split():
            if word in word2id:
                continue
            word2id[word] = len(word2id)
    #print(list(word2id.keys()))
    list_features=list(word2id.keys())

    # Bag of Wordsを作る
    bow_set = []
    for line in morphemes:
        bow = [0] * len(word2id)
        for word in line.split():
            try:
                bow[word2id[word]] += 1
            except:
                pass
        bow_set.append(bow)
    array_bow=np.array(bow_set)
    #print(np.array(bow_set), sep="\n")

    df_bow=pd.DataFrame(array_bow, columns=list_features)
    return df_bow, list_features
  

corpus_2=[
          '梅雨は嫌だ。早く雨がやんでほしい。そしたら絵を見に行こう。',
          '愛のケダモノ 音楽と絵 空想.',
          'カフェオレ レモン 心',
          '虎の子は虎だ アルコールの使いすぎで肌荒れ']

df_bow, list_features=Bag_of_Words(corpus_2)
df_bow

,梅雨は嫌だ。早く雨がやんでほしい。そしたら絵を見に行こう。,愛のケダモノ,音楽と絵,空想.,カフェオレ,レモン,心,虎の子は虎だ,アルコールの使いすぎで肌荒れ
0,1,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0
2,0,0,0,0,1,1,1,0,0
3,0,0,0,0,0,0,0,1,1




---



In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from IPython.display import HTML

def words_freq(corpus, corpus_all):
    pd.set_option('display.max_columns', None)
    pd.options.display.float_format = '{:0.2f}'.format

    df_BoW, list_features_BoW=Bag_of_Words(corpus)
    df_BoW_all, list_features_BoW_all=Bag_of_Words(corpus_all)
    #------------------------------------------------------------------------------------
    # --- BoW (Bag of Words) ---
    #------------------------------------------------------------------------------------

    print('--- BoW (Bag of Words) ---')
    # print(df_bow_focus)
    df_BoW_selected_by_focus=df_BoW_all.filter(items=list_features_BoW, axis=1)
    #------------------------------------------------------------------------------------
    # TF を計算してるところ (行方向の処理)
    #------------------------------------------------------------------------------------
    # 文書に含まれる単語の数をカウントする
    array_BoW=df_BoW.values
    number_of_words = np.sum(array_BoW, axis=1, keepdims=True)
    # print("number_of_words", number_of_words )
    # 文書の中での単語の頻度を計算する
    tf = array_BoW / number_of_words
    df_tf = pd.DataFrame(tf, columns=list_features_BoW)
    #------------------------------------------------------------------------------------
    # IDF を計算してるところ (列方向の処理)
    #------------------------------------------------------------------------------------
    #print('--- IDF (Inverse Document Frequency) ---')
    # 文書の数をカウントする
    number_of_docs = len(corpus_all)
    # その単語が一つでも含まれる文書の数をカウントする
    array_BoW_selected=df_BoW_selected_by_focus.values
    number_of_docs_contain_word = np.count_nonzero(array_BoW_selected, axis=0)
    # 単語の珍しさを計算する
    idf = np.log(number_of_docs / number_of_docs_contain_word)
    df_idf = pd.DataFrame([idf], columns=list_features_BoW)
    #------------------------------------------------------------------------------------
    # TF-IDF を計算してるところ
    #------------------------------------------------------------------------------------
    #print('--- TF-IDF ---')
    # TF と IDF をかける
    #
    tfidf = tf * idf
    df_tfidf = pd.DataFrame(tfidf, columns=list_features_BoW)
    df_results=pd.DataFrame( columns=["words", "sum_TF-IDF"])
    df_results["words"]=df_tfidf.columns.to_list()
    df_results["sum_TF-IDF"]=df_tfidf.sum().to_list()
    df_results=df_results.set_index("words")
    df_results.sort_values("sum_TF-IDF", ascending=False, inplace=True)

    return df_results

# corpus = [
#           'This is the first document.',
#           'This document is the second document.',
#           'And this is the third one.',
#           'Is this the first document?',
#           ]

# df_results=words_freq(corpus)
# HTML(df_results.to_html())

In [ ]:
from janome.tokenizer import Tokenizer
import re
t = Tokenizer()

def make_queue(list_text):
    list_sentence_ifidf=[]
    list_sentence_ifidf.clear()
    list_noun=[]
    list_noun.clear()
    for sent in list_text:
        #print(sent)
        if sent.startswith("私の性格タイプは"): #私の性格タイプは
            #print(sent)
            continue
        # Tokenizeで分割
        count_noun=0
        tokens=t.tokenize(sent)
#         for token in tokens:
#             #print(token.part_of_speech.split(','))
#             p = token.part_of_speech.split(',')
#             if p[0]=='名詞' and p[1]=='一般':
#                 list_noun.append(token.surface)
#                 count_noun+=1
#         one_sentence=' '.join(list_noun)
        sub_list_noun=[]
        sub_list_noun.clear()
        for token in tokens:
            #print(token.part_of_speech.split(','))
            p = token.part_of_speech.split(',')
            if p[0]=='名詞' and p[1]=='一般':
                if token.surface == 'ﾟ' or 'ー' in token.surface:
                    continue
                sub_list_noun.append(token.surface)
                count_noun+=1
        one_sentence=' '.join(sub_list_noun)
        list_noun.append(one_sentence)
        if count_noun==1:
            #print(token.surface)
            pass
        #
    return list_noun
    #return one_sentence
    
#list_sentence_ifidf_1=make_quee(list_text_1)

In [ ]:
list_sentence_tfidf_0=[]
list_sentence_tfidf_1=[]
list_sentence_tfidf_0.clear()
list_sentence_tfidf_1.clear()
list_sentence_all=[]
list_sentence_all.clear()

list_sentence_all=make_queue(list_text_all)
list_sentence_0=make_queue(list_text_0)
list_sentence_1=make_queue(list_text_1)
df_tfidf_0=words_freq(list_sentence_0, list_sentence_all)
df_tfidf_1=words_freq(list_sentence_1, list_sentence_all)

--- BoW (Bag of Words) ---


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


--- BoW (Bag of Words) ---


In [ ]:
HTML(df_tfidf_0[:15].to_html())

,sum_TF-IDF
words,
人,90.41
RT,77.83
自分,71.77
気温,42.48
キャラ,32.23
気,26.44
最高,26.31
世界,25.99
感じ,25.60


In [ ]:
HTML(df_tfidf_1[:15].to_html())

,sum_TF-IDF
words,
PS,54.98
AKATUBA,37.96
電車,34.26
月,30.36
おかげ,22.95
あと,22.41
クリスマス,20.75
最後,19.29
o,16.43
